In [159]:
#!git clone https://github.com/brewing-datacup/reto-colombia-2020

In [160]:
#!pip install patool

In [161]:
#!pip install pyunpack
#from pyunpack import Archive
#Archive("/content/reto-colombia-2020/data/colombia-data.7z").extractall("/content/drive/My Drive/Reto/")

In [162]:
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [163]:
clientes_df = pd.read_csv("/content/drive/My Drive/Reto/Input1_clientes_estructura.csv", sep =';')
ventas_df = pd.read_csv("/content/drive/My Drive/Reto/Input2_clientes_venta.csv", sep =';')
base_pred= pd.read_csv("/content/drive/My Drive/Reto/Input3_clientes_test.csv", sep=";")

In [164]:
ventas_df.replace({'Marca2': r'^Marca','Cupo2': r'^Cupo','CapacidadEnvase2': r'^CapacidadEnvase'}
                  , {'Marca2': 'M', 'Cupo2':'C','CapacidadEnvase2':'CE'}, regex=True,inplace=True)

In [165]:
ventas_df['Fecha'] = pd.to_datetime(ventas_df['Año'].astype(str)+ ventas_df['Mes'].apply(lambda x: '0'+str(x) if x<=9 else str(x)) +'01')
ventas_df=ventas_df[ventas_df['Volumen']>0]
ventas_df['Marca'] = ventas_df['Marca2']+'-'+ventas_df['Cupo2']+'-'+ventas_df['CapacidadEnvase2']
ventas_df.drop(columns=['Año','Mes','Marca2','Cupo2','CapacidadEnvase2'], inplace=True)

In [166]:
marcas_minimas =pd.DataFrame(ventas_df.groupby(['Marca'])['Marca'].count().sort_values())
marcas_minimas.columns=['Cantidad']
marcas_minimas.reset_index(inplace=True)
marcas= marcas_minimas[marcas_minimas['Cantidad']>=2000]['Marca'].to_list()
ventas_df=ventas_df[ventas_df['Marca'].isin(marcas)]
prueba = ventas_df

In [167]:
grouped_cliente = prueba.groupby(['Cliente','Fecha']).agg({'Volumen': ['sum','mean'],'disc': ['sum'],'nr': ['sum'],'Marca':['nunique']}).reset_index()
grouped_cliente.columns = ['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente']

In [168]:
grouped_cliente_marca = prueba.groupby(['Cliente','Fecha', 'Marca']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum']}).reset_index()
grouped_cliente_marca.columns = ['Cliente','Fecha','Marca','volumen_marca','descuento_marca','valor_marca']

In [169]:
prueba_2 = pd.merge(grouped_cliente, grouped_cliente_marca, left_on=['Cliente','Fecha'], right_on=['Cliente','Fecha'], how="left")

In [170]:
prueba_2['p_vol'] =prueba_2['volumen_marca']/prueba_2['volumen_cliente']
prueba_2['p_desc'] =prueba_2['descuento_marca']/prueba_2['descuento_cliente']
prueba_2['p_valor'] =prueba_2['valor_marca']/prueba_2['valor_cliente']

In [171]:
prueba_2.drop(columns=['volumen_marca','descuento_marca','valor_marca'], inplace=True)

In [172]:
prueba3= pd.pivot_table(prueba_2, values=['p_vol','p_desc','p_valor'], 
                         index=['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente'],
                         columns=['Marca'], aggfunc=np.mean).reset_index()

In [173]:
prueba3.columns = ['_'.join(col) for col in prueba3.columns]
prueba3.rename(columns={'Cliente_': 'Cliente', 
                        'Fecha_': 'Fecha',
                        'volumen_cliente_':'volumen_cliente',
                        'descuento_cliente_':'descuento_cliente',
                        'valor_cliente_':'valor_cliente',
                        'distinct_marca_cliente_':'distinct_marca_cliente'}, inplace=True)

In [174]:
prueba3.fillna(0, inplace=True)

In [175]:
ult_3_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-05-01'))]
ult_6_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-01-01'))]
ult_12_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-07-01'))]

In [176]:
ult_3_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-06-01'))]
ult_6_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-02-01'))]
ult_12_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-08-01'))]

In [177]:
columns = prueba3.columns.to_list()
columns.remove('Cliente')
columns.remove('Fecha') 

estaditicas = ['mean','median']

agg_group ={}

for i in columns :
    key=i
    agg_group[key] = estaditicas

In [178]:
ult_3_meses_train_agg = ult_3_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_train_agg.columns = ['_3_'.join(col) for col in ult_3_meses_train_agg.columns]
ult_3_meses_train_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_train_agg = ult_6_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_train_agg.columns = ['_6_'.join(col) for col in ult_6_meses_train_agg.columns]
ult_6_meses_train_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_train_agg = ult_12_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_train_agg.columns = ['_12_'.join(col) for col in ult_12_meses_train_agg.columns]
ult_12_meses_train_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [179]:
ult_3_meses_test_agg = ult_3_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_test_agg.columns = ['_3_'.join(col) for col in ult_3_meses_test_agg.columns]
ult_3_meses_test_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_test_agg = ult_6_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_test_agg.columns = ['_6_'.join(col) for col in ult_6_meses_test_agg.columns]
ult_6_meses_test_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_test_agg = ult_12_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_test_agg.columns = ['_12_'.join(col) for col in ult_12_meses_test_agg.columns]
ult_12_meses_test_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [180]:
clientes_df_dummies = pd.get_dummies(clientes_df)

In [181]:
base_train = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test.fillna(0, inplace=True)
base_train.fillna(0, inplace=True)

In [182]:
train_filtro=base_train['Cliente'].to_list()
test_filtro=base_test['Cliente'].to_list()

In [183]:
y_train = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-08-01')]
y_train = y_train[['Cliente','Marca']]
y_train = pd.get_dummies(y_train)

columns_t = y_train.columns.to_list()
columns_t.remove('Cliente') 

estadisticas_t = ['sum']

agg_group_train ={}

for i in columns_t :
    key=i
    agg_group_train[key] = estadisticas_t
    
y_train = y_train.groupby(['Cliente']).agg(agg_group_train).reset_index()
y_train.columns = ['_'.join(col) for col in y_train.columns]
y_train.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_train = y_train[y_train['Cliente'].isin(train_filtro)]

In [184]:
y_test = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-09-01')]
y_test = y_test[['Cliente','Marca']]
y_test = pd.get_dummies(y_test)

columns_te = y_test.columns.to_list()
columns_te.remove('Cliente') 

estadisticas_te = ['sum']

agg_group_test ={}

for i in columns_te :
    key=i
    agg_group_test[key] = estadisticas_te
    
y_test = y_test.groupby(['Cliente']).agg(agg_group_test).reset_index()
y_test.columns = ['_'.join(col) for col in y_test.columns]
y_test.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_test = y_test[y_test['Cliente'].isin(test_filtro)]

In [185]:
train_filtro2=y_train['Cliente'].to_list()
test_filtro2=y_test['Cliente'].to_list()

In [186]:
base_train = base_train[base_train['Cliente'].isin(train_filtro2)]
base_test = base_test[base_test['Cliente'].isin(test_filtro2)]

In [187]:
y_train.set_index('Cliente' , inplace = True)
y_train.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

In [188]:
base_train.set_index('Cliente' , inplace = True)

In [158]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import multiprocessing
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectFromModel

columns =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_train)

for i in columns :
  # create the classifier
  model=SelectFromModel(RandomForestClassifier(n_estimators = 100))
  y_base=y_train[i]
  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_scaled, y_base, test_size=0.3, random_state=8)
  model.fit(X_train, Y_train)
  cols = model.get_support(indices=True)
  selected_feat=(base_train.iloc[:,cols].columns.tolist())
  #selected_feat.append(base_train.columns[(model.get_support(indices=True))])
  best=model.transform(X_train)
  bestx=model.transform(X_validation)

  param_grid = {'n_estimators': [150],
              'max_features': [10, 15, 20],
              'max_depth'   : [None, 3, 10, 20],
              'criterion'   : ['gini', 'entropy']
             }
  w = 30
# Búsqueda por grid search con validación cruzada
# ==============================================================================
  clf = GridSearchCV(
        estimator  = RandomForestClassifier(class_weight={0: 1, 1: w},random_state = 123),
        param_grid = param_grid,
        scoring    = 'roc_auc',
        n_jobs     = multiprocessing.cpu_count() - 1,
        refit      = True,
        verbose    = 0,
        return_train_score = True)
  
  #y_base=y_train[i]
  #X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_scaled, y_base, test_size=0.3, random_state=8)
  # Train the model using the training sets
  clf.fit(best, Y_train)
  #y_pred = model.predict(X_test)
  
  fpr, tpr, thresholds = metrics.roc_curve(Y_validation, clf.predict_proba(bestx)[:,1])
  auc = metrics.roc_auc_score(Y_validation, clf.predict_proba(bestx)[:,1])
  print("AUC: " + str(i)+" "+ str(auc))
  
  with open("/content/drive/My Drive/Reto/smodel_2{}.pickle".format(i), "wb") as f:
    pickle.dump(clf, f) 
    pickle.dump(selected_feat, f)
  f.close()

AUC: M_20-C_3-CE_9 0.7826175533956412
AUC: M_16-C_2-CE_10 0.9123530489880345
AUC: M_9-C_3-CE_12 0.8660891816657158
AUC: M_38-C_2-CE_10 0.8742386478847276
AUC: M_39-C_2-CE_10 0.8748007811299887


In [189]:
columns =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]

In [190]:
# LOAD MODEL
#y_test.set_index('Cliente' , inplace = True)
#base_test.set_index('Cliente' , inplace = True)
y_test.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_test)

# j=0
# for i in columns: 
#    j=j+1
#    with open("/content/drive/My Drive/Reto/smodel_2{}.pickle".format(i), "rb") as f:
#      model = pickle.load(f)
#      base_test=base_test[selected_feat[j]]
#      base_test
#      #colsb=base_test.columns.to_list()
#      predicted= model.predict(base_test)
#      fpr, tpr, thresholds = metrics.roc_curve(y_test[i], model.predict_proba(base_test)[:,1])
#      auc = roc_auc_score(y_test[i], model.predict_proba(base_test)[:,1])
#      print("AUC: " + str(i)+" "+ str(auc))

for i in columns: 
   with open("/content/drive/My Drive/Reto/smodel_2{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     features = pickle.load(f)
     
     base=base_test[features]
     X_scaled = scaler.fit_transform(base)
     
     predicted= model.predict(X_scaled)

     auc = roc_auc_score(y_test[i], model.predict_proba(X_scaled)[:,1])
     print("AUC: " + str(i)+" "+ str(auc)) 
     CM = confusion_matrix(y_test[i], predicted)
     print(CM)

AUC: M_20-C_3-CE_9 0.7852365331688467
[[11159   544]
 [ 1296   661]]
AUC: M_16-C_2-CE_10 0.8706032146033628
[[13492     0]
 [  168     0]]
AUC: M_9-C_3-CE_12 0.8321091596288743
[[9169 1056]
 [1438 1997]]
AUC: M_38-C_2-CE_10 0.794619672627425
[[8220 2606]
 [ 926 1908]]
AUC: M_39-C_2-CE_10 0.8120346382446484
[[9388 2609]
 [ 489 1174]]


In [191]:
base_filtro=base_pred['Cliente'].to_list()
prueba4 = prueba3[prueba3['Cliente'].isin(base_filtro)]
clientes_df_dummies_2= clientes_df_dummies[clientes_df_dummies["Cliente"].isin(base_filtro)]

In [192]:
ult_3_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-07-01'))]
ult_6_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-03-01'))]
ult_12_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-09-01'))]

ult_3_meses_base_agg = ult_3_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_base_agg.columns = ['_3_'.join(col) for col in ult_3_meses_base_agg.columns]
ult_3_meses_base_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_base_agg = ult_6_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_base_agg.columns = ['_6_'.join(col) for col in ult_6_meses_base_agg.columns]
ult_6_meses_base_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_base_agg = ult_12_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_base_agg.columns = ['_12_'.join(col) for col in ult_12_meses_base_agg.columns]
ult_12_meses_base_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

base_fin = pd.merge(clientes_df_dummies_2,ult_12_meses_base_agg,left_on=['Cliente'], right_on=['Cliente'], how="left")
base_fin= pd.merge(base_fin, ult_6_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin= pd.merge(base_fin, ult_3_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin.set_index('Cliente' , inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [193]:
base_fin.fillna(0, inplace=True)

In [194]:
scaler = StandardScaler()
X_scaled1 = scaler.fit_transform(base_fin)

for i in columns: 
   with open("/content/drive/My Drive/Reto/smodel_2{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     selected_feat = pickle.load(f)

     base=base_fin[selected_feat]
     X_scaled1 = scaler.fit_transform(base)
     base_fin[i]=np.round(model.predict_proba(X_scaled1)[:,1],3)#df['column'].round(1)
base_predic=base_fin[columns]

In [195]:
#base_predic.reset_index(inplace=True)
base_predic.columns = ['Marca1',	'Marca2',	'Marca3',	'Marca_Inno1',	'Marca_Inno2']


In [196]:
base_predic.to_csv("/content/drive/My Drive/Reto/output4_2_2.csv")

In [197]:
base_predic

Marca1  Marca2  Marca3  Marca_Inno1  Marca_Inno2
Cliente                                                  
1         0.114   0.002   0.091        0.672        0.598
2         0.482   0.002   0.027        0.052        0.077
3         0.114   0.002   0.092        0.561        0.528
4         0.554   0.002   0.042        0.299        0.336
5         0.480   0.006   0.292        0.066        0.042
...         ...     ...     ...          ...          ...
20411     0.075   0.001   0.044        0.896        0.907
20414     0.084   0.001   0.148        0.835        0.799
20434     0.088   0.001   0.130        0.824        0.812
20462     0.089   0.001   0.134        0.798        0.820
20569     0.076   0.001   0.173        0.598        0.615

[9302 rows x 5 columns]